<a href="https://colab.research.google.com/github/idilhatipoglu/SE_Project/blob/main/streamlit_audio_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#import libraries 
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import IPython.display as ipd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext tensorboard
from sklearn.neighbors import KNeighborsClassifier


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
from sklearn.neighbors import KNeighborsClassifier

In [18]:
import pickle

In [2]:
#mounting the google drive we have datasets on as well as a csv we will need later on (all on github)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#new paths for cleaned datasets
path_train = 'ManualTrainClean/'
path_test = 'ManualTestClean/'
path_validate = 'ManualValidationClean/'


header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

import csv 
import os
import librosa
import numpy as np 


#We extract features from a spectogram: 
#Mel-frequency cepstral coefficients (MFCC), Spectral Centroid, Zero Crossing Rate, Chroma Frequencies, and Spectral Roll-off
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
insects = 'ManualTrainClean ManualValidationClean ManualTestClean'.split()
for g in insects:
    for filename in os.listdir(f'/content/drive/MyDrive/SE project/Data/Audio Data/Insect Audio_Classification/{g}'):
        songname = f'/content/drive/MyDrive/SE project/Data/Audio Data/Insect Audio_Classification/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)[0]
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {filename[0:14]}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

228

532

528

529

530

...

530

529

539

539

539

In [6]:
#Loading the created CSV data
import pandas as pd
df = pd.read_csv('dataset.csv')
df

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,Chorthippusbiguttulus112.wav,0.634230,0.007856,5265.892717,2632.058839,8146.886575,0.394584,-407.025665,-14.476236,38.443523,...,12.136890,-3.356176,10.180372,-5.155545,8.998631,-3.963912,9.788656,-5.747363,8.735406,Chorthippusbig
1,Brevisianabrevis50.wav,0.278553,0.023671,8360.393147,1613.225812,9603.858555,0.779192,-442.207611,-67.499908,81.449593,...,13.395598,-13.078304,29.403847,-6.787571,6.159097,1.122410,13.147902,-8.514930,9.527393,Brevisianabrev
2,Chorthippusbiguttulus140.wav,0.431901,0.002844,6995.820211,2203.414537,9286.489633,0.614703,-677.206055,-41.768703,72.509941,...,8.548782,-7.049221,11.121943,-8.727956,9.934090,-6.404820,8.529761,-5.373106,7.926888,Chorthippusbig
3,Brevisianabrevis201.wav,0.221812,0.022767,8711.206868,1226.801899,9622.668882,0.791720,-514.264099,-83.227180,94.395981,...,14.649964,-8.227493,14.580876,-7.115521,10.249490,-4.339907,12.998123,-5.216075,13.344728,Brevisianabrev
4,Chorthippusbiguttulus130.wav,0.599224,0.016785,4301.377385,2723.963251,7660.260922,0.317736,-330.437073,43.683323,35.710815,...,5.839953,-10.417289,4.927583,-9.593264,3.887088,-8.526044,5.485570,-9.100643,5.561647,Chorthippusbig
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,Platypleuraintercapedinis11.wav,0.348551,0.004804,7022.943665,2980.415392,9547.877223,0.657227,-517.678894,7.069996,70.585342,...,15.412003,-2.406152,14.015229,-3.562848,13.253787,-9.770341,11.422911,-5.495098,8.286010,Platypleuraint
338,Platypleuraplumosa40.wav,0.271010,0.100189,6541.875371,2382.375483,8228.981022,0.571111,-327.107605,-9.416759,101.048485,...,21.990911,4.219244,7.949275,-1.190679,7.639871,0.196287,6.967792,-1.914606,7.794403,Platypleuraplu
339,Platypleuraintercapedinis19.wav,0.196914,0.010828,8579.766074,2338.977888,10042.366499,0.845769,-459.888550,-29.674074,55.187992,...,1.496284,2.395417,9.689463,-1.595024,6.101700,-2.443599,9.673810,-4.216593,5.978100,Platypleuraint
340,Tettigoniaviridissima9.wav,0.165061,0.008520,8911.000315,904.673316,9446.097819,0.750728,-693.939148,-71.831085,94.227448,...,2.561816,-20.322771,7.056552,-26.854233,19.112560,-27.621887,11.518591,-20.756100,11.801855,Tettigoniaviri


In [7]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np 

#Data preprocessing: label encoding, feature scaling and data split into training and test set.
data = df.copy()
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into Training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,Chorthippusbiguttulus112.wav,0.634230,0.007856,5265.892717,2632.058839,8146.886575,0.394584,-407.025665,-14.476236,38.443523,...,12.136890,-3.356176,10.180372,-5.155545,8.998631,-3.963912,9.788656,-5.747363,8.735406,Chorthippusbig
1,Brevisianabrevis50.wav,0.278553,0.023671,8360.393147,1613.225812,9603.858555,0.779192,-442.207611,-67.499908,81.449593,...,13.395598,-13.078304,29.403847,-6.787571,6.159097,1.122410,13.147902,-8.514930,9.527393,Brevisianabrev
2,Chorthippusbiguttulus140.wav,0.431901,0.002844,6995.820211,2203.414537,9286.489633,0.614703,-677.206055,-41.768703,72.509941,...,8.548782,-7.049221,11.121943,-8.727956,9.934090,-6.404820,8.529761,-5.373106,7.926888,Chorthippusbig
3,Brevisianabrevis201.wav,0.221812,0.022767,8711.206868,1226.801899,9622.668882,0.791720,-514.264099,-83.227180,94.395981,...,14.649964,-8.227493,14.580876,-7.115521,10.249490,-4.339907,12.998123,-5.216075,13.344728,Brevisianabrev
4,Chorthippusbiguttulus130.wav,0.599224,0.016785,4301.377385,2723.963251,7660.260922,0.317736,-330.437073,43.683323,35.710815,...,5.839953,-10.417289,4.927583,-9.593264,3.887088,-8.526044,5.485570,-9.100643,5.561647,Chorthippusbig


In [8]:
#Loading a manually created table of all of the insects names and their family order to be able to group them
df2 = pd.read_csv('/content/drive/MyDrive/SE project/Data/Audio Data/Insect Audio_Classification/klasyfikacja - Arkusz1.csv')

In [9]:
def set_label(row):
    if (row["label"] == "Chorthippusbig") or (row["label"] == "Chorthippusbru") or (row["label"] == "Grylluscampest") or (row["label"] == "Nemobiussylves") or (row["label"] == "Oecanthuspellu") or (row["label"] == "Pholidopteragr") or  (row["label"] == "Pseudochorthip") or (row["label"] == "Roeselianaroes") or (row["label"] == "Tettigoniaviri"):
        return "Orthoptera"
    else:
        return "Cicadidae"

df = df.assign(order=df.apply(set_label, axis=1))

In [10]:
df.drop(columns=['label'], axis = 1, inplace = True)

In [11]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


data = df.copy()
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,order
0,Chorthippusbiguttulus112.wav,0.634230,0.007856,5265.892717,2632.058839,8146.886575,0.394584,-407.025665,-14.476236,38.443523,...,12.136890,-3.356176,10.180372,-5.155545,8.998631,-3.963912,9.788656,-5.747363,8.735406,Orthoptera
1,Brevisianabrevis50.wav,0.278553,0.023671,8360.393147,1613.225812,9603.858555,0.779192,-442.207611,-67.499908,81.449593,...,13.395598,-13.078304,29.403847,-6.787571,6.159097,1.122410,13.147902,-8.514930,9.527393,Cicadidae
2,Chorthippusbiguttulus140.wav,0.431901,0.002844,6995.820211,2203.414537,9286.489633,0.614703,-677.206055,-41.768703,72.509941,...,8.548782,-7.049221,11.121943,-8.727956,9.934090,-6.404820,8.529761,-5.373106,7.926888,Orthoptera
3,Brevisianabrevis201.wav,0.221812,0.022767,8711.206868,1226.801899,9622.668882,0.791720,-514.264099,-83.227180,94.395981,...,14.649964,-8.227493,14.580876,-7.115521,10.249490,-4.339907,12.998123,-5.216075,13.344728,Cicadidae
4,Chorthippusbiguttulus130.wav,0.599224,0.016785,4301.377385,2723.963251,7660.260922,0.317736,-330.437073,43.683323,35.710815,...,5.839953,-10.417289,4.927583,-9.593264,3.887088,-8.526044,5.485570,-9.100643,5.561647,Orthoptera


In [17]:
# from sklearn import svm

# clf = svm.SVC()
# clf.fit(X_train, y_train)

# predictions = clf.predict(X_test)

# score = clf.score(X_test, y_test)
# score

In [17]:
knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)
  
# accuracy on X_test
accuracy = knn.score(X_test, y_test)
accuracy

0.9130434782608695

In [19]:
filename = 'finalized_audio_classification.sav'
pickle.dump(knn, open(filename, 'wb'))